In [ ]:
labels, vectors = vsa.sample(5)
emp = vsa.empty(2, 10)
emp, vectors

In [ ]:
vsa.bind(vectors[0], vectors[1]), vsa.bundle(vectors[0], vectors[1])

In [ ]:
vsa.multibind(vectors), vsa.multiset(vectors, False)

In [ ]:
vsa.similarity(vectors[0], vectors[1])

In [ ]:
import torch
weights = torch.tensor([1,2,3,4,-1])
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))
# weights * vectors.transpose(-2,-1)

In [ ]:
weights = torch.tensor([[1,2,3,4,-1], [-1,-2,-3, -4, 1]])
vectors.unsqueeze(0).repeat(2,1,1)
torch.matmul(weights.type(torch.LongTensor), vectors.type(torch.LongTensor))

In [ ]:
from main import algo1, algo2, algo3
from vsa.vsa import VSA
from vsa.resonator import Resonator
import torch
import shutil

shutil.rmtree("tests/testing/")

VSA_MODE = "HARDWARE"
NUM_VEC_SUPERPOSED = 2
ALGO = "ALGO3" # ALGO1, ALGO2, ALGO3
TRIALS = 20    # for ALGO2
DIM = 2000
FACTORS = 5
CODEVECTORS = 10
CODEVECTORS : tuple = (3,3,3,10,2)
NOISE_LEVEL = 0.0  # compositional vector noise
ITERATIONS = 5000    # max number of iterations for factorization
NORMALIZE = True   # for SOFTWARE mode. Normalize the initial estimate and the input vector (when the input is a bundled vector)
ACTIVATION = 'EXPO'  # 'NONE', 'ABS', 'NONNEG'
RESONATOR_TYPE = "SEQUENTIAL" # "CONCURRENT", "SEQUENTIAL"
ARGMAX_ABS = True
REORDER_CODEBOOKS = False # True False
if VSA_MODE == 'HARDWARE':
    NORMALIZE = None
BATCH_SIZE = 1

vsa = VSA(
    root="tests/testing/",
    mode=VSA_MODE,
    dim=DIM,
    num_factors=FACTORS,
    num_codevectors=CODEVECTORS,
    seed = 0
)

rn = Resonator(vsa, type=RESONATOR_TYPE, activation=ACTIVATION, iterations=ITERATIONS, argmax_abs=ARGMAX_ABS)
ds = VSADataset("tests/testing/", 1, vsa, algo=ALGO, num_vectors_superposed=NUM_VEC_SUPERPOSED, noise=NOISE_LEVEL)
# Test sample
label = [(0, 2, 1, 1), (1, 2, 1, 5)]
# label = [(0, 0, 2, 3), (2, 0, 0, 3)]
vector = ds.lookup_algo3(label)

labels = [label]
vectors = vector.repeat(BATCH_SIZE, 1)

codebooks = None
orig_indices = None

if REORDER_CODEBOOKS:
    codebooks, orig_indices = rn.reorder_codebooks(codebooks)

init_estimates = rn.get_init_estimates(codebooks, BATCH_SIZE)
if NORMALIZE:
    init_estimates = vsa.normalize(init_estimates)

if (NUM_VEC_SUPERPOSED == 1):
    outcome, iter = rn(vectors, init_estimates, codebooks, orig_indices)
    # Make them the same format as multi-vector for easier parsing
    outcomes = [[outcome[x]] for x in range(BATCH_SIZE)]
    convergences = [1 if iter == ITERATIONS-1 else 0] * BATCH_SIZE
    iters = [iter] * BATCH_SIZE
else:
    if ALGO == "ALGO1":
        outcomes, convergences, iters = algo1(vsa, rn, vectors, init_estimates, codebooks, orig_indices, NORMALIZE)
    elif ALGO == "ALGO2":
        outcomes, convergences, iters = algo2(vsa, rn, vectors, DIM, FACTORS, codebooks, orig_indices, NORMALIZE)
    elif ALGO == "ALGO3":
        outcomes, convergences, iters = algo3(vsa, rn, vectors, NORMALIZE)


In [ ]:
print(labels)
print(outcomes)
print(iters)
